In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
def internal_marginalise(xrb_frac):

    lambda_G = 4264/14 * xrb_frac # rate of signal G (events per year)
    # lambda_S = 487.62516272856794
    lambda_S = 6368/3 * 0.21
    delta_t = 0.15/365.25   # uncertainty window in years
    # delta_t = 0.0416666666667/365.25   # uncertainty window in years
    num_simulations = 5000  # number of simulations

    def _internal_runner(times_1, times_2, delta_t):
        coincidences = 0
        
        for time_1 in times_1:
            diff_matrix_count = np.nansum(np.abs(times_2 - time_1) <= delta_t)
            
            if diff_matrix_count > 0:
                coincidences += 1
        
        return coincidences

    def simulate_events(lambda_G, lambda_S, delta_t):
        """Simulate the occurrence of events from signal G and signal S"""
        n_G = np.random.poisson(lambda_G)
        n_S = np.random.poisson(lambda_S)
        
        coincidences = 0
        
        times_G = np.random.uniform(0, 1, n_G)
        times_S = np.random.uniform(0, 1, n_S)
        
        # for time_S in times_S:
        #     diff_matrix_count = np.nansum(np.abs(times_G - time_S) <= delta_t)
            
        #     if diff_matrix_count > 0:
        #         coincidences += 1
        
        coincidences = _internal_runner(times_G, times_S, delta_t) + _internal_runner(times_S, times_G, delta_t)
        
        return coincidences/(len(times_G) + len(times_S))

    def perform_simulations(lambda_G, lambda_S, delta_t, num_simulations):
        """Run the simulation multiple times and calculate the probability of coincidences"""
        coincidences_count = 0
        
        for _ in range(num_simulations):
            coincidences = simulate_events(lambda_G, lambda_S, delta_t)
            coincidences_count += coincidences
        
        probability_of_coincidence = coincidences_count / num_simulations # Probability of coincidence (P-value)
        
        return probability_of_coincidence

    probability = perform_simulations(lambda_G, lambda_S, delta_t, num_simulations) # Run the simulations to calculate the probability of coincidences
    
    return probability


In [5]:
probs = []

xrb_fracs = np.linspace(0, 1, 101)
for frac in tqdm(xrb_fracs, desc="Calculating probabilities"):
    prob = internal_marginalise(frac)
    probs.append(prob)


Calculating probabilities:   0%|          | 0/101 [00:00<?, ?it/s]

Calculating probabilities: 100%|██████████| 101/101 [27:12<00:00, 16.16s/it]


In [6]:
import scipy.stats as stats

probby = np.trapz(probs, xrb_fracs)
confidence_level = 1- probby * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")

confidence_level = 1- probby #* (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)
print(f"The z-score for a {probby*100:.3f}% confidence level is: {z_score:.3f}")

The z-score for a 15.322% confidence level is: 3.312
The z-score for a 15.322% confidence level is: 1.428


In [ ]:
probby

In [ ]:
probs[5], probs[95] 

In [ ]:

confs = [probs[5], probs[95]]

for c in confs:
    confidence_level = 1- c * (1 - 0.99394678)
    z_score = stats.norm.ppf((1 + confidence_level) / 2)
    print(f"The z-score for a {confidence_level*100:.3f}% confidence level is: {z_score:.3f}")

confidence_level = 1- 0.15321393130481006 * (1 - 0.99394678)
z_score = stats.norm.ppf((1 + confidence_level) / 2)

# # Output the result
print(f"The z-score for a {confidence_level*100:.3f}% confidence level is: {z_score:.3f}")

In [ ]:
100 - 99.907, (1 - 0.99394678)*100

In [ ]:
print(f"{(1-0.99394678)*100:.2f}% false alarm probability")

In [ ]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
h70 = cosmo.H0.value / 70

vol = cosmo.comoving_volume(0.028).value
# vol = cosmo.luminosity_distance(0.028).value**3 * 4 * np.pi / 3

vol

In [ ]:
rate_cc = 9.1e-5 / mpc**3 * h70**3

In [ ]:
9.1e-5 / 7698878.1351534575**3

In [ ]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# --- 1. Define cosmology ---
H0 = 70
Om0 = 0.3
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

# --- 2. Inputs ---
R_num = (6.1)*1e-5  # volumetric SN rate, quoted as [SNe yr^-1 Mpc^-3 h_70^3]
z_max = 0.028

# Fraction of sky covered (set to 1 for full sky)
f_sky = 1.0

# --- 3. Compute comoving volume ---
V_c = cosmo.comoving_volume(z_max)  # full-sky comoving volume

V_survey = V_c * f_sky             # restrict to survey area if needed

# --- 4. Rest-frame number per year ---
N_rest = R_num * V_survey.value   # SNe / yr (rest-frame)

print(f"Comoving volume: {V_survey:.3e}")
print(f"Expected SNe per year (rest-frame): {N_rest/(1+0):.2f}")


In [ ]:
from astropy import constants as const
from scipy.integrate import quad

# Define differential comoving volume element per unit z (full sky)
def dVdz(z):
    return cosmo.differential_comoving_volume(z).to(u.Mpc**3 / u.sr) * (4*np.pi * u.sr)

# Integrand: rate * dV/dz / (1+z)
def integrand(z):
    return (R_num * dVdz(z).value) / (1+z)

N_obs, _ = quad(integrand, 0, z_max)

# If you want to scale by f_sky < 1
N_obs *= f_sky

print(f"Expected SNe per OBSERVER year: {N_obs:.2f}")

In [ ]:
# SNe yr^-1 Mpc^-3 h_70^3

0.7**(-3)

In [ ]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u

# Define your cosmology
H0 = 70  # km/s/Mpc
cosmo = FlatLambdaCDM(H0=H0, Om0=0.3)

# Example inputs
R = 0.91e-4  # [SNe / yr / Mpc^3 / h70^3]
h70 = H0 / 70  # =1 for H0=70

r_comoving = cosmo.comoving_distance(0.028).value  # in Mpc
V_comoving = (4/3) * np.pi * r_comoving**3

In [ ]:
# Convert R to SNe/yr/Mpc^3 in your cosmology
R_corrected = R / h70**3

# Apply time dilation (observer frame)
N_obs_per_year = R_corrected * V_comoving / (1+0.028)

print(f"Redshift z = {0.028:.3f}")
print(f"Comoving volume = {V_comoving:.3e}")
print(f"Expected observed SN per year = {N_obs_per_year:.3f}")

In [ ]:
# volumetric_sn_rate.py
import numpy as np
from scipy.integrate import quad
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# ---------- USER INPUTS ----------
H0 = 70.0  # km/s/Mpc
Om0 = 0.3
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

R0 = 1e-4            # SNe / yr / Mpc^3  (source-frame) at z=0
z_max = 0.019         # integrate up to this redshift
evolution = "power"  # choices: "none" or "power"
k = 1.5              # power-law index for R(z) = R0 * (1+z)**k if evolution == "power"
# ---------------------------------

# define R(z)
def R_of_z(z):
    if evolution == "none":
        return R0
    elif evolution == "power":
        return R0 * (1.0 + z)**k
    else:
        raise ValueError("Unknown evolution model")

# integrand: R(z)/(1+z) * dV_c/dz * 4pi
# astropy's differential_comoving_volume returns units of Mpc3 / sr
def integrand(z):
    # differential comoving volume per steradian:
    dVc_dz = cosmo.differential_comoving_volume(z)  # astropy Quantity: Mpc3 / sr
    # R(z) is in SNe / yr / Mpc^3 (source frame). Convert to Quantity for safety:
    Rz = R_of_z(z) * (u.Unit("1/(yr Mpc3)"))
    # integrand value: (Rz/(1+z)) * dVc_dz * 4*pi  -> units: SNe / yr
    val = (Rz / (1.0 + z)) * dVc_dz * (4.0 * np.pi * u.sr)
    return val.to("1/yr").value  # return plain float in SNe / yr

# perform integral
N_obs, err = quad(integrand, 0.0, z_max, epsabs=0, epsrel=1e-4)
print(f"Observed SNe per year up to z={z_max:.2f}: {N_obs:.5g}  (approx)")

# Optional: show contribution per redshift slice (binned)
z_edges = np.linspace(0, z_max, 101)
z_centers = 0.5 * (z_edges[:-1] + z_edges[1:])
dN_dz = np.array([integrand(z) for z in z_centers])
dV_slice = np.diff(z_edges)
N_bins = np.sum(dN_dz * dV_slice)  # approximate Riemann sum approximation
print(f"Riemann-sum approx: {N_bins:.5g}  (should be close to quad result)")


In [ ]:
1e-4 * 4 * np.pi / 3